<a href="https://githubtocolab.com/kaust-halo/geeet/blob/master/examples/notebooks/04_TSEB_crop_water_use.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

## Use the TSEB model and upscale to daily ET (mm/day)

Here we apply the TSEB model to the image asset produced in the previous notebook (`03_prepare_ECMWF_L8_inputs_for_TSEB.ipynb`). We then use an upscaling model to estimate daily ET in mm/day. 

> note: currently the TSEB model in geeet is still experimental/under development. 

In [ ]:
# Use the following line to install geeet if needed:
#!pip install git+https://github.com/kaust-halo/geeet
# Use the following line to install geemap if needed:
#!pip install geemap
import ee
ee.Initialize()

Load the inputs image generated in the [previous notebook](https://github.com/kaust-halo/geeet/blob/main/examples/notebooks/03_prepare_ECMWF_L8_inputs_for_TSEB.ipynb) and run the TSEB model:

In [2]:
inputs_img = ee.Image('projects/geeet-public/assets/TSEB_sample_inputs')
from geeet.tseb import tseb_series
et = tseb_series(inputs_img, zU=10, zT=2)

Note that the output (LE) is instantaneous ET in $W/m2$. We can use the [Jackson et al. (1983)](<https://doi.org/10.1016/0378-3774(83)90095-1>) irradiance model for upscaling this value into daily average ET:
$$
ET_d \approx ET_i R
$$
where R is the ratio of instantaneous to daily radiation and can be obtained as:
$$
R =  \frac{2N}{2.45 \times 10^6 \pi \times sin(\pi \times t/N)}
$$
where N is the number of seconds between sunset and sunrise, and t is the number of seconds between sunrise and the observation time. We also apply a 1/2.45 scaling factor to convert the values to mm/day. 



This is available in the `geeet.solar.rad_ratio` function:

In [3]:
from geeet.solar import rad_ratio
def add_ET_mm_band(img):
    R = rad_ratio(img)
    LE = img.select('LE')
    ETd = LE.multiply(R).rename('ETd')
    return img.addBands(ETd)

et = add_ET_mm_band(et)

Finally, we export the output as an asset. This will take a while, as we are exporting it at a 30m scale. You can visualize it in an interactive map with this [code editor script](https://code.earthengine.google.com/?scriptPath=users%2Flopezvoliver%2Fgeeet%3Atseb_sample_outputs_vis), or at the end of this notebook using `geemap`. 

In [4]:
# Template cell to export the output as an asset
lonlat=[38.25,30.25]
geom = ee.Geometry.Point(lonlat[0], lonlat[1])
region = geom.buffer(10000) # 10-km radius around the point 

task = ee.batch.Export.image.toAsset(image=et,\
                                     description='TSEB_outputs_export',\
                                     assetId='projects/geeet-public/assets/TSEB_sample_outputs',\
                                     region=region,\
                                     scale=30)
task.start()

Interactive map (requires `geemap`). 

> n.b. here we load the pre-processed outputs available as an asset. If you want, you can comment the first line so that you actually use the `et` image created above - but it will be much slower to display as it will be running the computations "on-the-fly". 

In [ ]:
et = ee.Image('projects/geeet-public/assets/TSEB_sample_outputs')  # comment this line if you want to use the actual et image created above. Here we are loading a pre-computed asset. 

import geemap
import geemap.colormaps as cm
lonlat=[38.25,30.25]
Map = geemap.Map(center=[lonlat[1], lonlat[0]], zoom=12)
ndvi_pal = cm.palettes.ndvi
pal = cm.palettes.YlOrRd
pal_viridis = cm.palettes.viridis
magPal=['f1eef6', 'd7b5d8', 'df65b0', 'dd1c77', '980043']
purplePal=['edf8fb', 'bfd6e8', '9cacd2', '8a7cba', '87489f', '810f7c']

Map.addLayer(et.select('Hc'), {'min':0, 'max':500, 'palette':magPal}, 'H from canopy (W/m2)', False)
Map.addLayer(et.select('Hs'), {'min':0, 'max':500, 'palette':magPal}, 'H from soil (W/m2)', False)
Map.addLayer(et.select('G'), {'min':0, 'max':500, 'palette':magPal}, 'Ground heat flux (W/m2)', False)
Map.addLayer(et.select('Rn'), {'min':0, 'max':500, 'palette':magPal}, 'Net radiation (W/m2)')
Map.addLayer(et.select('LEc'), {'min':0, 'max':500, 'palette':magPal}, 'LE from canopy (W/m2)')
Map.addLayer(et.select('LEs'), {'min':0, 'max':500, 'palette':magPal}, 'LE from soil (W/m2)')
Map.addLayer(et.select('LE'), {'min':0, 'max':500, 'palette':magPal}, 'Total LE (W/m2)')
Map.addLayer(et.select('ETd'), {'min':0, 'max':10, 'palette':purplePal}, 'ET (mm/day)')